# 02b – 3. Experiment: Stage-2-Light-Fine-Tuning: MCoRec-fokussierter Trainingsmix

## Motivation

`02a` zeigte, dass ein zweites Fine-Tuning mit LR 5·10⁻⁵ und 30 k Schritten
das Modell überanpasst. Dieser Ansatz testet eine noch konservativere Variante:

| Parameter | Stage-2 (`02a`) | Stage-2-Light (`02b`) |
|-----------|-----------------|----------------------|
| Lernrate | 5·10⁻⁵ | **1·10⁻⁵** (5× kleiner) |
| Schritte | 30 000 | **5 000** |
| MCoRec-Anteil | ~20 % | **~70 %** (`--mcorec_mode heavy`) |
| Warmup | 3 000 Steps | **500 Steps** |

Hypothese: Weniger Schritte + stärkerer MCoRec-Fokus könnte Überanpassung verhindern
und gleichzeitig die Domänenanpassung verbessern.

## Ergebnis (Vorschau)

Auch Stage-2-Light ist **schlechter** als BL4 (WER +0.032–0.033, JER +0.016).
BL4 bleibt das beste Modell – weiteres Fine-Tuning wird nicht weiterverfolgt.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.

## 1 – Setup: Arbeitsverzeichnis & Imports

In [1]:
import os, sys
from pathlib import Path

# Path-Objekt für komfortables Pfad-Handling
project_baseline_path = Path("/home/josch080/Projektgruppe/mcorec_baseline")
os.chdir(project_baseline_path)
print("CWD:", os.getcwd())

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if str(project_baseline_path) not in sys.path:
    sys.path.append(str(project_baseline_path))


CWD: /home/josch080/Projektgruppe/mcorec_baseline


## 2 – GPU-Auswahl

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 3 – CUDA-Verifikation

In [3]:
import torch
print("n_gpu:", torch.cuda.device_count())
# sollte 1 ausgeben

n_gpu: 1


## 4 – Stage-2-Light-Training

Gegenüber `02a` sind drei Parameter verändert: Lernrate (5× kleiner),
Schrittanzahl (6× weniger) und MCoRec-Gewichtung (von ~20 % auf ~70 %).

**Hinweis:** `subprocess.run(cmd)` ist bereits ausgeführt; der Checkpoint liegt unter
`model-bin/avsr_cocktail_mcorec_stage2_light_lr1e-5_5k/checkpoint5000`.

In [4]:
cmd = [
    sys.executable, "script/train.py",
    "--streaming_dataset", # Daten on-the-fly laden
    "--include_mcorec", # MCoRec in den Trainingsmix aufnehmen
    "--mcorec_mode", "heavy",           # MCoRec stark gewichten: ~70 % statt ~20 % im Mix
    "--batch_size", "4",
    "--max_steps", "5000",              # Stark reduziert gegenüber Stage-2 (30k): minimales Nachtraining
    "--gradient_accumulation_steps", "2", # Effektive Batch-Größe = 4 × 2 = 8
    "--save_steps", "1000", # Checkpoint alle 1000 Schritte
    "--eval_steps", "1000", # Validation alle 1000 Schritte
    "--log_interval", "25",
    "--learning_rate", "1e-5",          # 5× kleiner als Stage-2 (5e-5)
    "--warmup_steps", "500",            # Kurzer Warmup passend zur geringen Schrittzahl
    "--checkpoint_name", "avsr_cocktail_mcorec_stage2_light_lr1e-5_5k",
    "--model_name_or_path", "./model-bin/avsr_cocktail_mcorec_finetune", # Startpunkt: BL4
    "--output_dir", "./model-bin",
    "--report_to", "none", # Kein externes Logging (z.B. W&B)
]
print(" ".join(cmd)) # Vollständigen Befehl zur Verifikation ausgeben


/home/josch080/Projektgruppe/mcorec_train/bin/python script/train.py --streaming_dataset --include_mcorec --mcorec_mode heavy --batch_size 4 --max_steps 5000 --gradient_accumulation_steps 2 --save_steps 1000 --eval_steps 1000 --log_interval 25 --learning_rate 1e-5 --warmup_steps 500 --checkpoint_name avsr_cocktail_mcorec_stage2_light_lr1e-5_5k --model_name_or_path ./model-bin/avsr_cocktail_mcorec_finetune --output_dir ./model-bin --report_to none


In [5]:
import subprocess

# Training starten – Ausgabe erscheint direkt im Notebook
subprocess.run(cmd)

Loading pretrained model from ./model-bin/avsr_cocktail_mcorec_finetune
Loading MCoRec dataset
map_datasets
 {'lrs2': {'probabilities': 0.1, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 10
}), 'valid': None}}, 'vox2': {'probabilities': 0.05, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 53
}), 'valid': None}}, 'avyt': {'probabilities': 0.1, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 16
}), 'valid': None}}, 'avyt-mix': {'probabilities': 0.05, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 664
}), 'valid': None}}, 'mcorec': {'probabilities': 0.7, 'dataset': {'train': IterableDataset({
    features: ['label', 'length', 'sample_id', 'video'],
    num_shards: 48
}), 'valid': IterableDataset({
    features: ['label', 'length', 's

/home/josch080/Projektgruppe/mcorec_baseline/src/custom_trainer.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `AVSRTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Could not estimate the number of tokens of the input, floating-point operations will not be computed
  0%|          | 25/5000 [00:24<37:58,  2.18it/s]  

{'loss': 9.1956, 'grad_norm': 22.156843185424805, 'learning_rate': 4.4e-07, 'epoch': 0.01}


  1%|          | 50/5000 [00:38<40:36,  2.03it/s]

{'loss': 12.3982, 'grad_norm': 22.16065216064453, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.01}


...

Too many dataloader workers: 10 (max is dataset.num_shards=3). Stopping 7 dataloader workers.
100%|██████████| 5000/5000 [55:47<00:00,  2.38it/s]

{'eval_loss': 27.72670555114746, 'eval_runtime': 187.3396, 'eval_samples_per_second': 20.775, 'eval_steps_per_second': 5.194, 'epoch': 1.0}


100%|██████████| 5000/5000 [55:56<00:00,  1.49it/s]


{'train_runtime': 3356.0327, 'train_samples_per_second': 11.919, 'train_steps_per_second': 1.49, 'train_loss': 10.243281088256836, 'epoch': 1.0}


CompletedProcess(args=['/home/josch080/Projektgruppe/mcorec_train/bin/python', 'script/train.py', '--streaming_dataset', '--include_mcorec', '--mcorec_mode', 'heavy', '--batch_size', '4', '--max_steps', '5000', '--gradient_accumulation_steps', '2', '--save_steps', '1000', '--eval_steps', '1000', '--log_interval', '25', '--learning_rate', '1e-5', '--warmup_steps', '500', '--checkpoint_name', 'avsr_cocktail_mcorec_stage2_light_lr1e-5_5k', '--model_name_or_path', './model-bin/avsr_cocktail_mcorec_finetune', '--output_dir', './model-bin', '--report_to', 'none'], returncode=0)

## 5 – Inference-Setup

In [6]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

## 6 – Modell-Definitionen

BL4 als Referenz und der neue Stage-2-Light-Checkpoint.

In [11]:
MODELS = {
     # BL4: Referenzmodell
    "cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
        "out": "output_avsr_cocktail_finetuned",
    },
    # Stage-2-Light: BL4 + 5k weitere Schritte mit MCoRec-Heavy-Mix
    "cocktail_stage2_light": {
    "model_type": "avsr_cocktail",
    "chkpt": "model-bin/avsr_cocktail_mcorec_stage2_light_lr1e-5_5k/checkpoint-5000",
    "out": "output_avsr_cocktail_stage2_light",
    },
}

## 7 – Sessions & Experimente

Gleiches 5-Session-Subset wie bei den vorherigen Experimenten für direkte Vergleichbarkeit.
Zwei Experimente mit den bisher besten Hyperparametern (beam=8/12, len=20).

In [12]:
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

In [18]:
EXPERIMENTS = {
    "E14_stage2_light_bs8_len20": {
    "base_model": "cocktail_stage2_light",
    "beam_size": 8,
    "max_length": 20,
    "comment": "Stage-2-Light, beam=8, len=20",
    },
    
    "E15_stage2_light_bs12_len20": {
    "base_model": "cocktail_stage2_light",
    "beam_size": 12,
    "max_length": 20,
    "comment": "Stage-2-Light, beam=12, len=20",
    },
}


## 8 – Inference

2 Experimente × 5 Sessions = 10 Läufe.

In [14]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E14_stage2_light_bs8_len20
  base_model      = cocktail_stage2_light
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_stage2_light_lr1e-5_5k/checkpoint-5000
  beam_size       = 8
  max_length      = 20
  output_dir_name = output_E14_stage2_light_bs8_len20
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = Stage-2-Light, beam=8, len=20
Loading avsr_cocktail model...
Loading model from model-bin/avsr_cocktail_mcorec_stage2_light_lr1e-5_5k/checkpoint-5000
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:43<03:37, 43.42s/it]


Processing speakers:  33%|███▎      | 2/6 [02:04<04:21, 65.43s/it]


Processing speaker spk_2 track 1 of 3: 0it [00:00, ?it/s]

Processing speaker spk_2 track 2 of 3: 100%|██████████| 13/13 [00:42<00:00,  3.27s/it]

Processing speakers:  50%|█████     | 3/6 [03:26<03:38, 72.94s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 26/26 [00:43<00:00,  1.67s/it]

Processing speakers:  20%|██        | 1/5 [00:57<03:49, 57.37s/it]


Processing speakers:  40%|████      | 2/5 [02:10<03:20, 66.85s/it]


Processing speakers:  60%|██████    | 3/5 [03:02<01:59, 59.96s/it]


Processing speakers:  80%|████████  | 4/5 [04:25<01:08, 68.89s/it]


Processing speakers: 100%|██████████| 5/5 [05:08<00:00, 61.70s/it]


## 9 – Evaluation & Aggregation

In [15]:
# Ergebnisse auswerten und an gemeinsame CSV anhängen
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)



########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 10 – Vergleich Stage-2-Light vs. BL4

Stage-2-Light-Ergebnisse (E1x) werden mit BL4-Ergebnissen (E0x) bei gleichen
Hyperparametern verglichen. Positive Δ-Werte = Stage-2-Light schlechter als BL4.

In [24]:
import pandas as pd

dev_df = pd.read_csv("results_dev_subset_by_session.csv")

# Stage-2-ähnliche Experimente (E1*) und Finetuned-Experimente (E0*)
stage2_like = dev_df[dev_df["model"].str.startswith("E1")].copy()
ft_df = dev_df[dev_df["model"].str.startswith("E0")].copy()

def parse_beam_len(model_name: str):
    parts = model_name.split("_")
    beam_part = next(p for p in parts if p.startswith("bs"))
    len_part = next(p for p in parts if p.startswith("len"))
    beam = int(beam_part.replace("bs", ""))
    length = int(len_part.replace("len", ""))
    return beam, length

# Hyperparameter-Spalten für beide DataFrames befüllen
for df in [stage2_like, ft_df]:
    df[["beam_size", "max_length"]] = df["model"].apply(
        lambda m: pd.Series(parse_beam_len(m))
    )

# Über Sessions mitteln
stage2_agg = (
    stage2_like
    .groupby(["model", "beam_size", "max_length"])[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

ft_agg = (
    ft_df
    .groupby(["model", "beam_size", "max_length"])[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

# Stage-2-Light und BL4 per (beam_size, max_length) zusammenführen
comparison = pd.merge(
    stage2_agg,
    ft_agg,
    on=["beam_size", "max_length"],
    how="left",
    suffixes=("_stage2", "_ft"),
)

# Δ berechnen: positiv = Stage-2-Light schlechter als BL4
comparison["Δ_WER_stage2_minus_ft"] = (
    comparison["avg_speaker_wer_stage2"] - comparison["avg_speaker_wer_ft"]
)
comparison["Δ_JER_stage2_minus_ft"] = (
    comparison["avg_joint_error_stage2"] - comparison["avg_joint_error_ft"]
)

# Nur E14 und E15 behalten (es gibt ggf. weitere E1x-Einträge aus 02a)
light_mask = comparison["model_stage2"].isin([
    "E14_stage2_light_bs8_len20",
    "E15_stage2_light_bs12_len20",
])

comparison_light = comparison[light_mask].copy()

# Lesbarere Variant-Labels für die Ausgaben
comparison_light["variant"] = comparison_light["model_stage2"].map({
    "E14_stage2_light_bs8_len20":  "Stage-2-light (beam=8, len=20)",
    "E15_stage2_light_bs12_len20": "Stage-2-light (beam=12, len=20)",
})

# Relevante Spalten in sinnvoller Reihenfolge
comparison_light = comparison_light[
    [
        "variant",
        "beam_size",
        "max_length",
        "avg_speaker_wer_stage2",
        "avg_joint_error_stage2",
        "avg_speaker_wer_ft",
        "avg_joint_error_ft",
        "Δ_WER_stage2_minus_ft",
        "Δ_JER_stage2_minus_ft",
    ]
]

display(comparison_light)


,variant,beam_size,max_length,avg_speaker_wer_stage2,avg_joint_error_stage2,avg_speaker_wer_ft,avg_joint_error_ft,Δ_WER_stage2_minus_ft,Δ_JER_stage2_minus_ft
4,"Stage-2-light (beam=8, len=20)",8,20,0.528553,0.340469,0.495798,0.324091,0.032755,0.016378
5,"Stage-2-light (beam=12, len=20)",12,20,0.527217,0.339800,0.495416,0.323900,0.031801,0.015900


## 11 – Interpretation

| Konfiguration | WER BL4 | WER Stage-2-Light | Δ WER | JER BL4 | JER Stage-2-Light | Δ JER |
|---------------|---------|-------------------|-------|---------|-------------------|-------|
| beam=8,  len=20 | 0.4958 | 0.5286 | +0.033 | 0.3241 | 0.3405 | +0.016 |
| beam=12, len=20 | 0.4954 | 0.5272 | +0.032 | 0.3239 | 0.3398 | +0.016 |

Stage-2-Light ist geringfügig besser als Stage-2 aus `02a` (Δ WER ~+0.032 vs. ~+0.036),
bleibt aber in beiden Konfigurationen **schlechter als BL4**.

**Fazit:** Auch mit aggressiverem MCoRec-Fokus und sehr niediger Lernrate
lässt sich BL4 durch weiteres Fine-Tuning nicht übertreffen.
BL4 (`avsr_cocktail_mcorec_finetune`) bleibt das Referenzmodell.
Fine-Tuning-Experimente werden nicht weiterverfolgt.
